# Data Cleaning Notebook
Here I created a ready to use notebook to clean an awful data.

---
## Data Preparation

First, I'll load every modules that are needed and will be used during the analysis process. And to prepare > pre-process the data.


--- 

Let's check out if the pre-requisite library was installed on the systems or not.

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install seaborn
!pip install nltk
!pip install wordcloud
!pip install scikit-learn

In [1]:
#---------------------------------------------------------------
# Make sure if the packages needed alraedy installed on systems

import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline

# A lot of modules was loaded here, I loaded it for the convenience even if not everything will be used.
#--------------------------------------------------------------------------------------------------------

Then, I will load the data and get some basic information in the loaded dataframe.

Checking out the directory.

In [10]:
import glob
cwd = os.getcwd()
listdir = os.listdir()
listdir

['.ipynb_checkpoints',
 'Amazon Sale Report.csv',
 'Customer Segmentation_basic info phase.ipynb',
 'Data Cleaning.ipynb']

In [13]:
#-----------------------------------------------
# Reading the data by setting the datapath first

#-----------------------------------------------------------
# Getting the current working directory (root for notebook) and pass it into data path
#cwd = os.getcwd()
datapath = cwd # >> use this only if the data outside cwd+ '\E-commerce data' #Set the working directory path
#print(datapath)

#------------------------------------------------------------------
# Checking file encoding so pandas can decode it into the dataframe
with open(datapath + '\Amazon Sale Report.csv') as f: # < specify the filename inside paramater
    display(f)
    display("^^^^^^^ Based on the output above, we can see the encoding used by the datafile.")
    
#----------------------------------
#We can start to read the file now

pd.set_option('display.max_columns', None) 
df_init = pd.read_csv(datapath + '\Amazon Sale Report.csv', encoding='cp1252')
display(df_init)
display(df_init.info)
display(df_init.columns)

#Let's move on to the pre-processing stage
#--------------------------------------

<_io.TextIOWrapper name='D:\\Data Satria\\Python Notebook Learn\\November 2024\\Amazon Sales\\Amazon Sale Report.csv' mode='r' encoding='cp1252'>

'^^^^^^^ Based on the output above, we can see the encoding used by the datafile.'

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,NaN,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,128970,406-6001380-7673107,05-31-22,Shipped,Amazon,Amazon.in,Expedited,JNE3697,JNE3697-KR-XL,kurta,XL,B098112V2V,Shipped,1,INR,517.00,HYDERABAD,TELANGANA,500013.0,IN,NaN,False,NaN,False
128971,128971,402-9551604-7544318,05-31-22,Shipped,Amazon,Amazon.in,Expedited,SET401,SET401-KR-NP-M,Set,M,B09VC6KHX8,Shipped,1,INR,999.00,GURUGRAM,HARYANA,122004.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False
128972,128972,407-9547469-3152358,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0157,J0157-DR-XXL,Western Dress,XXL,B0982YZ51B,Shipped,1,INR,690.00,HYDERABAD,TELANGANA,500049.0,IN,NaN,False,NaN,False
128973,128973,402-6184140-0545956,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,XS,B0894Y2NJQ,Shipped,1,INR,1199.00,Halol,Gujarat,389350.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False


<bound method DataFrame.info of          index             Order ID      Date                        Status  \
0            0  405-8078784-5731545  04-30-22                     Cancelled   
1            1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2            2  404-0687676-7273146  04-30-22                       Shipped   
3            3  403-9615377-8133951  04-30-22                     Cancelled   
4            4  407-1069790-7240320  04-30-22                       Shipped   
...        ...                  ...       ...                           ...   
128970  128970  406-6001380-7673107  05-31-22                       Shipped   
128971  128971  402-9551604-7544318  05-31-22                       Shipped   
128972  128972  407-9547469-3152358  05-31-22                       Shipped   
128973  128973  402-6184140-0545956  05-31-22                       Shipped   
128974  128974  408-7436540-8728312  05-31-22                       Shipped   

       Fulfilment S

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

We clearly have known the basic information of the data above, now let's start to the next phase, pre-processing it.

Create a new dataframe with columns that we'll use

In [14]:
df_use = df_init[['Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country']]
df_use.head(2)

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN


In [26]:
#Rename the columns into a better one.

column_indices = [5,6,7,9,10,11,12]
new_names = ['Shipment Type','Quantity', 'Currency', 'City','State', 'Postal Code', 'Country']
old_names = df_use.columns[column_indices]
df_use.rename(columns=dict(zip(old_names, new_names)), inplace=True)
df_use.head(2)

,Order ID,Date,Status,Fulfilment,Sales Channel,Shipment Type,Quantity,Currency,Amount,City,State,Postal Code,Country
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN


Convert the date datatype and explore the basic info

In [27]:
#------------------------------------------------
# Converting the dtype of the date into datetime
display(df_use['Date'].info())
df_use['Date'] = pd.to_datetime(df_init['Date'])
display(df_use['Date'].info())

#----------------------------------------------------------------------------------------
# Now, we're checking the information of the data, wheter there are null/NaN values or not.
col_info = pd.DataFrame(df_use.dtypes).T.rename(index={0: 'dtype each Columns'})
null_info_nb = pd.DataFrame(df_use.isnull().sum()).T.rename(index={0:'Null Values'})
null_info_pct = pd.DataFrame(df_use.isnull().sum()/df_use.shape[0]*100).T.rename(index={0:'Null Values in %'})

#display(col_info)
#display(null_info_nb)
#display(null_info_pct)

info_list = [col_info, null_info_nb, null_info_pct]
data_basic_info = pd.concat(info_list, keys=map(lambda x: x.index.name, info_list))
display(data_basic_info)

#-----------------------------------------------------------------------------------

<class 'pandas.core.series.Series'>
RangeIndex: 128975 entries, 0 to 128974
Series name: Date
Non-Null Count   Dtype 
--------------   ----- 
128975 non-null  object
dtypes: object(1)
memory usage: 1007.7+ KB


None

<class 'pandas.core.series.Series'>
RangeIndex: 128975 entries, 0 to 128974
Series name: Date
Non-Null Count   Dtype         
--------------   -----         
128975 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1007.7 KB


None

Order ID            Date  Status Fulfilment  \
None dtype each Columns   object  datetime64[ns]  object     object   
     Null Values               0               0       0          0   
     Null Values in %        0.0             0.0     0.0        0.0   

                        Sales Channel  Shipment Type Quantity  Currency  \
None dtype each Columns         object        object    int64    object   
     Null Values                     0             0        0      7795   
     Null Values in %              0.0           0.0      0.0  6.043807   

                           Amount      City     State Postal Code   Country  
None dtype each Columns   float64    object    object     float64    object  
     Null Values             7795        33        33          33        33  
     Null Values in %    6.043807  0.025586  0.025586    0.025586  0.025586

As we can see from the table above, there are a lot of null values inside the data. We're trying to clean it now (we'll either impute it or drop the values), as well as assigning a correct datatype in the incorrect one.
But we need to deep analysis regarding the data, let's check the data once again.

In [43]:
null_rows = df_use.query('Currency != Currency or Amount != Amount')

display(null_rows.head(5))

,Order ID,Date,Status,Fulfilment,Sales Channel,Shipment Type,Quantity,Currency,Amount,City,State,Postal Code,Country


As we can see in the table above, we can't just remove the null values because it contains important data. let's just impute it to zero value.

In [38]:
df_use[['Currency', 'Amount']] = df_use[['Currency', 'Amount']].fillna(0)
null_rows = df_use.query('Currency != Currency or Amount != Amount')

display(null_rows.head(5))

,Order ID,Date,Status,Fulfilment,Sales Channel,Shipment Type,Quantity,Currency,Amount,City,State,Postal Code,Country


In [42]:
null_rows = df_use.query('`Currency` == 0' or '`Amount` = 0')

display(null_rows.count())

Order ID          7795
Date              7795
Status            7795
Fulfilment        7795
Sales Channel     7795
Shipment Type     7795
Quantity          7795
Currency          7795
Amount            7795
City              7793
State             7793
Postal Code       7793
Country           7793
dtype: int64

Seems if there are around 7,7k of cancelled order in the data. <br>
Let's check the city, state null values.

In [44]:
null_rows = df_use.query('City != City or State != State')

display(null_rows.head(5))

,Order ID,Date,Status,Fulfilment,Sales Channel,Shipment Type,Quantity,Currency,Amount,City,State,Postal Code,Country
1871,404-0566904-4825137,2022-04-29,Shipped,Amazon,Amazon.in,Expedited,1,INR,493.0,NaN,NaN,NaN,NaN
1872,404-0566904-4825137,2022-04-29,Shipped,Amazon,Amazon.in,Expedited,1,INR,458.0,NaN,NaN,NaN,NaN
8752,406-4003386-8768363,2022-04-25,Shipped,Amazon,Amazon.in,Expedited,1,INR,432.0,NaN,NaN,NaN,NaN
11215,402-0107720-7057168,2022-04-23,Shipped,Amazon,Amazon.in,Expedited,1,INR,654.0,NaN,NaN,NaN,NaN
13252,407-4532637-8415521,2022-04-22,Cancelled,Merchant,Amazon.in,Standard,0,INR,380.0,NaN,NaN,NaN,NaN


It seems that the other data are available, we might just impute the value as 'Other'

In [45]:
df_use[['City', 'State', 'Postal Code', 'Country']] = df_use[['City', 'State', 'Postal Code', 'Country']].fillna('Other')
other_val = df_use.query("City == 'Other'")

display(other_val.head(5))

,Order ID,Date,Status,Fulfilment,Sales Channel,Shipment Type,Quantity,Currency,Amount,City,State,Postal Code,Country
1871,404-0566904-4825137,2022-04-29,Shipped,Amazon,Amazon.in,Expedited,1,INR,493.0,Other,Other,Other,Other
1872,404-0566904-4825137,2022-04-29,Shipped,Amazon,Amazon.in,Expedited,1,INR,458.0,Other,Other,Other,Other
8752,406-4003386-8768363,2022-04-25,Shipped,Amazon,Amazon.in,Expedited,1,INR,432.0,Other,Other,Other,Other
11215,402-0107720-7057168,2022-04-23,Shipped,Amazon,Amazon.in,Expedited,1,INR,654.0,Other,Other,Other,Other
13252,407-4532637-8415521,2022-04-22,Cancelled,Merchant,Amazon.in,Standard,0,INR,380.0,Other,Other,Other,Other


Let's check out the whole data info again.

In [46]:
col_info = pd.DataFrame(df_use.dtypes).T.rename(index={0: 'dtype each Columns'})
null_info_nb = pd.DataFrame(df_use.isnull().sum()).T.rename(index={0:'Null Values'})
null_info_pct = pd.DataFrame(df_use.isnull().sum()/df_use.shape[0]*100).T.rename(index={0:'Null Values in %'})

#display(col_info)
#display(null_info_nb)
#display(null_info_pct)

info_list = [col_info, null_info_nb, null_info_pct]
data_basic_info = pd.concat(info_list, keys=map(lambda x: x.index.name, info_list))
display(data_basic_info)


Order ID            Date  Status Fulfilment  \
None dtype each Columns   object  datetime64[ns]  object     object   
     Null Values               0               0       0          0   
     Null Values in %        0.0             0.0     0.0        0.0   

                        Sales Channel  Shipment Type Quantity Currency  \
None dtype each Columns         object        object    int64   object   
     Null Values                     0             0        0        0   
     Null Values in %              0.0           0.0      0.0      0.0   

                          Amount    City   State Postal Code Country  
None dtype each Columns  float64  object  object      object  object  
     Null Values               0       0       0           0       0  
     Null Values in %        0.0     0.0     0.0         0.0     0.0

There's no more null values, how about the duplicate? <br>
And change postal code into string.

In [49]:
# df_use[COL.isin(COL[COL.duplicated()])].sort_values("ID")
df_use[df_use.duplicated(keep=False)]

,Order ID,Date,Status,Fulfilment,Sales Channel,Shipment Type,Quantity,Currency,Amount,City,State,Postal Code,Country
117,403-0104882-7373900,2022-04-30,Cancelled,Amazon,Amazon.in,Expedited,1,INR,807.0,HIMMATNAGAR,Gujarat,383001.0,IN
118,403-0104882-7373900,2022-04-30,Cancelled,Amazon,Amazon.in,Expedited,1,INR,807.0,HIMMATNAGAR,Gujarat,383001.0,IN
166,407-3742007-3242711,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,1,INR,517.0,HYDERABAD,TELANGANA,500086.0,IN
167,407-3742007-3242711,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,1,INR,517.0,HYDERABAD,TELANGANA,500086.0,IN
306,408-3870331-6287517,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,1,INR,399.0,Dehradun,UTTARAKHAND,248001.0,IN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128802,402-4418047-0659535,2022-06-01,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,1,INR,799.0,AGARTALA,TRIPURA,799006.0,IN
128907,403-6757403-6097100,2022-05-31,Cancelled,Amazon,Amazon.in,Expedited,0,0,0.0,GREATER NOIDA,UTTAR PRADESH,201306.0,IN
128908,403-6757403-6097100,2022-05-31,Cancelled,Amazon,Amazon.in,Expedited,0,0,0.0,GREATER NOIDA,UTTAR PRADESH,201306.0,IN
128952,404-8878574-4524358,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,1,INR,301.0,Rishra Hooghly,WEST BENGAL,712249.0,IN


Let's just drop every duplicated data!

In [59]:

display('Before :', df_use.shape[0])
df_ready = df_use.drop_duplicates()
display("After :", df_ready.shape[0])

removed_dupe = df_use.shape[0] - df_ready.shape[0]
display
display('Duplicates Removed :', removed_dupe)

'Before :'

128975

'After :'

127515

'Duplicates Removed :'

1460

Now just repair the rest to make the data looks better.

In [77]:
df_ready['Postal Code'] = df_ready['Postal Code'].str.replace('.0','')
df_ready['City'] = df_ready['City'].str.title()
df_ready['State'] = df_ready['State'].str.title()

display(df_ready.head(5))
display(df_ready.info())

,Order ID,Date,Status,Fulfilment,Sales Channel,Shipment Type,Quantity,Currency,Amount,City,State,Postal Code,Country
0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,0,INR,647.62,Mumbai,Maharashtra,400081,IN
1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,1,INR,406.00,Bengaluru,Karnataka,560085,IN
2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,1,INR,329.00,Navi Mumbai,Maharashtra,410210,IN
3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,0,INR,753.33,Puducherry,Puducherry,605008,IN
4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,1,INR,574.00,Chennai,Tamil Nadu,600073,IN


<class 'pandas.core.frame.DataFrame'>
Index: 127515 entries, 0 to 128974
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Order ID        127515 non-null  object        
 1   Date            127515 non-null  datetime64[ns]
 2   Status          127515 non-null  object        
 3   Fulfilment      127515 non-null  object        
 4   Sales Channel   127515 non-null  object        
 5   Shipment Type   127515 non-null  object        
 6   Quantity        127515 non-null  int64         
 7   Currency        127515 non-null  object        
 8   Amount          127515 non-null  float64       
 9   City            127515 non-null  object        
 10  State           127515 non-null  object        
 11  Postal Code     127515 non-null  object        
 12  Country         127515 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(10)
memory usage: 13.6+ MB


None

The data is ready to use. Let's save it :)

In [78]:
df_ready.to_csv('Amazon Sales_cleaned.csv')

In [79]:
dfcheck = pd.read_csv('Amazon Sales_cleaned.csv')
dfcheck

,Unnamed: 0,Order ID,Date,Status,Fulfilment,Sales Channel,Shipment Type,Quantity,Currency,Amount,City,State,Postal Code,Country
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,0,INR,647.62,Mumbai,Maharashtra,400081,IN
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,1,INR,406.00,Bengaluru,Karnataka,560085,IN
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,1,INR,329.00,Navi Mumbai,Maharashtra,410210,IN
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,0,INR,753.33,Puducherry,Puducherry,605008,IN
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,1,INR,574.00,Chennai,Tamil Nadu,600073,IN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127510,128970,406-6001380-7673107,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,1,INR,517.00,Hyderabad,Telangana,500013,IN
127511,128971,402-9551604-7544318,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,1,INR,999.00,Gurugram,Haryana,122004,IN
127512,128972,407-9547469-3152358,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,1,INR,690.00,Hyderabad,Telangana,500049,IN
127513,128973,402-6184140-0545956,2022-05-31,Shipped,Amazon,Amazon.in,Expedited,1,INR,1199.00,Halol,Gujarat,389350,IN
